<a href="https://colab.research.google.com/github/nisha1365/GEN-AI-POC/blob/main/vaccine_topic_name_generation_120.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import boto3
import json
import os

In [ ]:
bedrock = boto3.client(

service_name='bedrock-runtime',

region_name='us-west-2',

endpoint_url='https://bedrock.us-west-2.amazonaws.com'

)


claude_config = {
    "max_tokens_to_sample": 1000,
    "temperature": 0.2,
    "stop_sequences": ["Human:"]
    }
textGenModelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

In [ ]:
bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

# Topic Name Generation:

In [ ]:
# Load your dataset
import pandas as pd
topics = pd.read_csv("C:\\Users\\2211592\\Downloads\\topic_info_Pediatric_120topic.csv")
topics

,Topic,Count,Name,Representation,Representative_Docs
0,-1,14340,-1_vaccination_vaccine_vaccinate_measles,"['vaccination', 'vaccine', 'vaccinate', 'measl...",['unvaccinated child getting sick & amp ; vacc...
1,0,5088,0_autism_vaccination_kid_mortality,"['autism', 'vaccination', 'kid', 'mortality', ...",['safest vaccine vaccine “ lockdown ” american...
2,1,2055,1_measles_mmr_case_catching,"['measles', 'mmr', 'case', 'catching', 'outbre...",['3 million unvaccinated child risk catching m...
3,2,2028,2_covid_pregnant_risk_vaccine,"['covid', 'pregnant', 'risk', 'vaccine', 'vacc...",['neonatal outcome covid-19 vaccination pregna...
4,3,1452,3_tetanus_shot_hope_stitch,"['tetanus', 'shot', 'hope', 'stitch', 'wound',...","['get tetanus shot', 'would get tetanus shot',..."
...,...,...,...,...,...
115,114,15,114_nasty_flusnotfunny_reception_spray,"['nasty', 'flusnotfunny', 'reception', 'spray'...",['flu cause lot nasty symptom one want see chi...
116,115,15,115_infertile_paternity_settling_widespread,"['infertile', 'paternity', 'settling', 'widesp...",['careful `` woman new york started lying vacc...
117,116,15,116_corynebacterium_diphtheriae_tonsil_bacterial,"['corynebacterium', 'diphtheriae', 'tonsil', '...",['diphtheria serious infection caused strain b...
118,117,15,117_hookworm_ringworm_worm_infections,"['hookworm', 'ringworm', 'worm', 'infections',...","[""'m joining war hookworm tetanus side hookwor..."


len(topics)

In [ ]:
topics = topics.iloc[1: , :]
topics

,Topic,Count,Name,Representation,Representative_Docs
1,0,5088,0_autism_vaccination_kid_mortality,"['autism', 'vaccination', 'kid', 'mortality', ...",['safest vaccine vaccine “ lockdown ” american...
2,1,2055,1_measles_mmr_case_catching,"['measles', 'mmr', 'case', 'catching', 'outbre...",['3 million unvaccinated child risk catching m...
3,2,2028,2_covid_pregnant_risk_vaccine,"['covid', 'pregnant', 'risk', 'vaccine', 'vacc...",['neonatal outcome covid-19 vaccination pregna...
4,3,1452,3_tetanus_shot_hope_stitch,"['tetanus', 'shot', 'hope', 'stitch', 'wound',...","['get tetanus shot', 'would get tetanus shot',..."
5,4,758,4_rsv_pfizer_preterm_pregnant,"['rsv', 'pfizer', 'preterm', 'pregnant', 'neon...",['pfizer failed disclose risk preterm birth ne...
...,...,...,...,...,...
115,114,15,114_nasty_flusnotfunny_reception_spray,"['nasty', 'flusnotfunny', 'reception', 'spray'...",['flu cause lot nasty symptom one want see chi...
116,115,15,115_infertile_paternity_settling_widespread,"['infertile', 'paternity', 'settling', 'widesp...",['careful `` woman new york started lying vacc...
117,116,15,116_corynebacterium_diphtheriae_tonsil_bacterial,"['corynebacterium', 'diphtheriae', 'tonsil', '...",['diphtheria serious infection caused strain b...
118,117,15,117_hookworm_ringworm_worm_infections,"['hookworm', 'ringworm', 'worm', 'infections',...","[""'m joining war hookworm tetanus side hookwor..."


In [ ]:
import time

In [ ]:
def label_tweets(docs):

    #global bedrock, bedrock_runtime

    # if(step == 1):

    system_prompt =  f"""Tweet: {docs}\n You act like a Topic Name Generator. You check the docs and generate a proper topic name which is suitable to represent the docs.

    To do this task you follow the below instructions one by one:

    Instruction 1: Do not generate a topic name by picking random words from the docs. The topic name should be proper and meaningful and contexual.
    Instruction 2: Do not write "Topic Name: " just give the topic name you have generated only. No extra sentence or words other than that is allowed.
    Instruction 3: Do not give any explanation, the focus is on generating the name of topic only.
    Instruction 4: If you have multiple suggestions as a Topic_Name for a single docs, then provide the most deserving topic's name according to you.
    Instruction 5: Do not write based on what you are generating the topic name. Just generate the topics

    For Example:

    -['American Academy Pediatrics ( 67,000 physician ) state â€œ The AAP recommends COVID-19 vaccination eligible child adolescent â€ care patient , know data . COVID kill child https//t.co/uFApgY9Ewk', 'American Academy Pediatrics ( 67,000 physician ) state â€œ The AAP recommends COVID-19 vaccination eligible child adolescent â€ care patient , know data . COVID kill child https//t.co/uFApgY9Ewk', 'American Academy Pediatrics ( 67,000 physician ) state â€œ The AAP recommends COVID-19 vaccination eligible child adolescent â€ care patient , know data . COVID kill child https//t.co/uFApgY9Ewk']
    Pediatric COVID-19 Vaccination

    -["Two dos MMR vaccination help stop child becoming seriously unwell measles . It 's important catchâ€¦", 'Two dos MMR vaccination help stop child becoming seriously unwell measles . For informationâ€¦', 'Two dos MMR vaccination help stop child becoming seriously unwell measles . For informationâ€¦']
    Measles Vaccination
    """

    # else:

    #     pgtx = f"Concatenate all the points : {pgtx}"

    prompt = f"\n\nHuman:{system_prompt}\n\nAssistant:"

    # print(prompt)

    body = json.dumps({"prompt": prompt,

                 "max_tokens_to_sample":4096,

                 "temperature":0.5,

                 "top_k":250,

                 "top_p":0.5,

                 "stop_sequences":[]

                  })

    modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider

    accept = 'application/json'

    contentType = 'application/json'

    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())

    proper_names =  response_body.get('completion')

    # ghj = ghj.replace(" Here is a summary without significant information loss:","")

    proper_names = proper_names.strip()

    return proper_names

topics['Topic_Name'] = pd.NA

# Initialize variables
batch_size = len(topics)
count = 0

# Iterate over the dataset to label the tweets
for index, row in topics.iterrows():
    docs = row['Representative_Docs']
    label = label_tweets(docs)
    topics.at[index, 'Topic_Name'] = label
    print(f"Name Generated {index+1}")

    # Sleep for 5 seconds to avoid hitting API rate limits
    time.sleep(1)

    # Increment count and break the loop if 1000 records are labeled
    count += 1
    if count >= batch_size:
        break
#return my_vaccine_data

C:\Users\2211592\AppData\Local\Temp\ipykernel_25004\3369640231.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topics['Topic_Name'] = pd.NA


Name Generated 2
Name Generated 3
Name Generated 4
Name Generated 5
Name Generated 6
Name Generated 7
Name Generated 8
Name Generated 9
Name Generated 10
Name Generated 11
Name Generated 12
Name Generated 13
Name Generated 14
Name Generated 15
Name Generated 16
Name Generated 17
Name Generated 18
Name Generated 19
Name Generated 20
Name Generated 21
Name Generated 22
Name Generated 23
Name Generated 24
Name Generated 25
Name Generated 26
Name Generated 27
Name Generated 28
Name Generated 29
Name Generated 30
Name Generated 31
Name Generated 32
Name Generated 33
Name Generated 34
Name Generated 35
Name Generated 36
Name Generated 37
Name Generated 38
Name Generated 39
Name Generated 40
Name Generated 41
Name Generated 42
Name Generated 43
Name Generated 44
Name Generated 45
Name Generated 46
Name Generated 47
Name Generated 48
Name Generated 49
Name Generated 50
Name Generated 51
Name Generated 52
Name Generated 53
Name Generated 54
Name Generated 55
Name Generated 56
Name Generated 57


In [ ]:
topics.to_csv('topics_name_120.csv', index=False)